In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [8]:
import pandas as pd
import numpy as np

def baseline_linear_pricing(df, alpha=0.05, base_price=20):
    df = df.copy()
    df['OccupancyRate'] = df['Occupancy'] / df['Capacity']
    df['Price_Model1'] = base_price + alpha * df['OccupancyRate'] * base_price
    return df[['ID', 'LastUpdatedDate', 'LastUpdatedTime', 'Price_Model1']]

def demand_based_pricing(df, alpha=0.4, beta=0.3, gamma=0.2, delta=0.1, epsilon=0.15, base_price=20):
    df = df.copy()
    df['OccupancyRate'] = df['Occupancy'] / df['Capacity']
    df['QueueNorm'] = df['QueueLength'] / df['QueueLength'].max()
    df['TrafficNorm'] = df['TrafficConditionNearby'] / df['TrafficConditionNearby'].max()
    vehicle_weights = {'car': 1.0, 'bike': 0.7, 'truck': 1.2}
    df['VehicleWeight'] = df['VehicleType'].map(vehicle_weights)
    df['Demand'] = (
        alpha * df['OccupancyRate'] +
        beta * df['QueueNorm'] -
        gamma * df['TrafficNorm'] +
        delta * df['IsSpecialDay'] +
        epsilon * df['VehicleWeight']
    )
    min_demand = df['Demand'].min()
    max_demand = df['Demand'].max()
    df['NormalizedDemand'] = (df['Demand'] - min_demand) / (max_demand - min_demand + 1e-5)
    lambda_factor = 0.4
    df['Price_Model2'] = base_price * (1 + lambda_factor * df['NormalizedDemand'])
    return df[['ID', 'LastUpdatedDate', 'LastUpdatedTime', 'Price_Model2']]

df = pd.read_csv("/kaggle/input/dataset-csv/dataset.csv")
numeric_columns = ['Occupancy', 'Capacity', 'QueueLength', 'TrafficConditionNearby', 'IsSpecialDay']
df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')

model1_prices = baseline_linear_pricing(df)
model2_prices = demand_based_pricing(df)

final_df = df[['ID', 'LastUpdatedDate', 'LastUpdatedTime']].copy()
final_df = final_df.merge(model1_prices, on=['ID', 'LastUpdatedDate', 'LastUpdatedTime'])
final_df = final_df.merge(model2_prices, on=['ID', 'LastUpdatedDate', 'LastUpdatedTime'])

final_df.to_csv("final_pricing_output.csv", index=False)
print("✅ Output saved to final_pricing_output.csv")

✅ Output saved to final_pricing_output.csv
